In [70]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import pickle
import requests
import torch
import torch.nn as nn
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from typing import List, Optional, Tuple, Union


nltk.download('stopwords')
nltk.download('punkt')

df_original = pd.read_csv('/root/projects/personal/comitiva_esperanca/data/ingestion/RAW/noticias_cepea.csv', sep=',', on_bad_lines='skip', encoding='utf-8')
df = df_original.copy()
df = df[df['data'] != 'data']
df = df.dropna()
df
padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

df


# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)

df_noticias = df.copy()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
class LIABertClassifier(nn.Module):
    def __init__(self,model,num_labels):
        super(LIABertClassifier,self).__init__()
        self.bert = model
        self.config = model.config
        self.num_labels = num_labels
        self.cls = nn.Linear(self.config.hidden_size,200)
        self.dropout = nn.Dropout(p=0.5)
        self.cls2 = nn.Linear(768,num_labels)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        ) ->Tuple[torch.Tensor]:

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs[0][:,0,:]
        prediction = self.dropout(sequence_output)
        prediction = self.cls2(prediction)
        return prediction

In [72]:

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')


import torch

device = torch.device('cpu')

# Crie uma instância de seu modelo
modelo_soja = LIABertClassifier(model, 3)
modelo_soja.load_state_dict(torch.load("/root/projects/personal/comitiva_esperanca/data/model/modelo_redeneural_bert_soja.pt"))

modelo_soja.to(device)

## carregar o modelo
modelo_soja.eval()

## realizar o predict
frase = "O demanda da soja começa a cair e os preços começam a cair também"
frase = preprocess_text(frase)
frase = tokenizer.encode(frase, return_tensors="pt")
outputs = modelo_soja(frase)

if outputs.argmax(axis=1).cpu().numpy().tolist()[0] == 0:
    print('Negativa')
elif outputs.argmax(axis=1).cpu().numpy().tolist()[0] == 1:
    print('Positiva')
else:
    print('Neutra')

## mostre a probabilidade (de 0 a 100%) de cada classe
probs = outputs.softmax(axis=1).cpu().detach().numpy().tolist()[0]
print(f"Negativa (%):", probs[0]*100)
print(f"Positiva (%):", probs[1]*100)
print(f"Neutra (%):", probs[2]*100)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Negativa
Negativa (%): 99.56121444702148
Positiva (%): 0.11925846338272095
Neutra (%): 0.3195257391780615


In [83]:
## predict para o dataframe
df_noticias_predict = df_noticias[:10].copy()
df_noticias_predict['noticia'] = df_noticias_predict['noticia'].apply(preprocess_text)
df_noticias_predict['noticia'] = df_noticias_predict['noticia'].apply(lambda x: tokenizer.encode(x, return_tensors="pt"))
df_noticias_predict['predict'] = df_noticias_predict['noticia'].apply(lambda x: modelo_soja(x).argmax(axis=1).cpu().numpy().tolist()[0])
df_noticias_predict['prob_negativa'] = df_noticias_predict['noticia'].apply(lambda x: modelo_soja(x).softmax(axis=1).cpu().detach().numpy().tolist()[0][0])
df_noticias_predict['prob_positiva'] = df_noticias_predict['noticia'].apply(lambda x: modelo_soja(x).softmax(axis=1).cpu().detach().numpy().tolist()[0][1])
df_noticias_predict['prob_neutra'] = df_noticias_predict['noticia'].apply(lambda x: modelo_soja(x).softmax(axis=1).cpu().detach().numpy().tolist()[0][2])
df_noticias_predict['predict'] = df_noticias_predict['predict'].apply(lambda x: 'Negativa' if x == 0 else 'Positiva' if x == 1 else 'Neutra')
df_noticias_predict.drop(columns=['noticia'], inplace=True)
df_noticias_predict

,data,titulo,url,predict,prob_negativa,prob_positiva,prob_neutra
12,10/04/2023,SOJA/CEPEA: Preço da soja sobe no BR; exportaç...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Positiva,0.005282,0.945870,0.048849
17,29/04/2021,ESPECIAL/CEPEA: Cepea aplica pesquisa relacion...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Neutra,0.226557,0.199788,0.573655
28,26/04/2021,SOJA/CEPEA: Demandas interna e externa aquecid...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Positiva,0.001163,0.991637,0.007200
37,17/05/2021,SOJA/CEPEA: Alta externa eleva movimentação no...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Neutra,0.020046,0.001406,0.978548
55,19/12/2022,SOJA/CEPEA: Preço do farelo de soja segue em a...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Positiva,0.003674,0.964559,0.031766
62,15/03/2021,SOJA/CEPEA: Valores internos oscilam com força,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Neutra,0.015884,0.281273,0.702843
66,12/03/2021,FRANGO/CEPEA: Poder de compra do produtor aume...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Neutra,0.008508,0.052981,0.938511
76,17/02/2021,SOJA/CEPEA: Colheita avança e pressiona cotaçã...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Neutra,0.332580,0.010639,0.656781
92,25/01/2021,SOJA/CEPEA: Preços do farelo voltam a subir no...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Negativa,0.627063,0.010819,0.362118
119,29/03/2021,SOJA/CEPEA: Produtor se retrai e valores volta...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,Positiva,0.001866,0.816472,0.181662


In [84]:
df_total = df_original[1:].copy()
# join df_noticias_predict
df_total = df_total.merge(df_noticias_predict, how='right', on=['data', 'titulo', 'url'])
df_total



,data,titulo,url,noticia,predict,prob_negativa,prob_positiva,prob_neutra
0,10/04/2023,SOJA/CEPEA: Preço da soja sobe no BR; exportaç...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 10/04/2023 – As cotações internas da so...",Positiva,0.005282,0.945870,0.048849
1,10/04/2023,SOJA/CEPEA: Preço da soja sobe no BR; exportaç...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 10/04/2023 – As cotações internas da so...",Positiva,0.005282,0.945870,0.048849
2,29/04/2021,ESPECIAL/CEPEA: Cepea aplica pesquisa relacion...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 29/04/2021 – Pesquisadores da área de P...",Neutra,0.226557,0.199788,0.573655
3,26/04/2021,SOJA/CEPEA: Demandas interna e externa aquecid...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 26/04/2021 – Os preços da soja estão em...",Positiva,0.001163,0.991637,0.007200
4,17/05/2021,SOJA/CEPEA: Alta externa eleva movimentação no...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 17/05/2021 – Os preços externos da soja...",Neutra,0.020046,0.001406,0.978548
5,19/12/2022,SOJA/CEPEA: Preço do farelo de soja segue em a...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 19/12/2022 – Com as demandas interna e ...",Positiva,0.003674,0.964559,0.031766
6,15/03/2021,SOJA/CEPEA: Valores internos oscilam com força,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 15/03/2021 – Os preços da soja registra...",Neutra,0.015884,0.281273,0.702843
7,12/03/2021,FRANGO/CEPEA: Poder de compra do produtor aume...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 12/03/2021 – Neste início de março, os ...",Neutra,0.008508,0.052981,0.938511
8,17/02/2021,SOJA/CEPEA: Colheita avança e pressiona cotaçã...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 17/02/2021 – A diminuição das chuvas na...",Neutra,0.332580,0.010639,0.656781
9,25/01/2021,SOJA/CEPEA: Preços do farelo voltam a subir no...,https://www.cepea.esalq.usp.br/br/diarias-de-m...,"Cepea, 25/01/2021 – Consumidores brasileiros d...",Negativa,0.627063,0.010819,0.362118


In [85]:

def publicar_noticia(row):
    url = 'http://20.75.1.155/news'
    data = {
        'title': row['titulo'],
        'newsContent': row['noticia'],
        "commodityType": "Soja",
        "source": "CEPEA",
        "positiveSentiment": row['prob_positiva'],
        "neutralSentiment": row['prob_neutra'],
        "negativeSentiment": row['prob_negativa'],
        "finalSentiment": row['predict'],
    }
    response = requests.post(url, json=data)
    print(response.status_code, response.text)

df_total.apply(publicar_noticia, axis=1)

201 {"statusCode":201,"message":null,"entity":{"id":"5ee022f4-d974-4ade-bdbc-18ff2f890c57","title":"SOJA/CEPEA: Preço da soja sobe no BR; exportação de março é recorde para o mês","newsContent":"Cepea, 10/04/2023 – As cotações internas da soja têm registrado variações positivas nos últimos dias, resultado do avanço no preço externo, da baixa produtividade das lavouras argentinas e, sobretudo, da maior demanda externa pelo grão brasileiro. De acordo com a Secex, o Brasil exportou em março/23 mais que o dobro do volume escoado em fevereiro/23 e 8,8% a mais que o de março/22, totalizando 13,27 milhões de toneladas. Essa é a maior quantidade de soja enviada ao exterior desde maio/21 e um recorde para março. Fonte: Cepea (","publicationDate":"0001-01-01T00:00:00","commodityType":"Soja","source":"CEPEA","positiveSentiment":0.9458699226379395,"neutralSentiment":0.04884850233793259,"negativeSentiment":0.0052815992385149,"finalSentiment":"Positiva"},"validationErrors":[]}
201 {"statusCode":201,

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
dtype: object

: 